# Purpose


This notebook demonstrates the data pipeline from raw tables to analytical datasets. At the end of this activity, train & test data sets are created from raw data.



## Imports

In [75]:
from pprint import pprint
import os
import os.path as op
import shutil

# standard third party imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.options.mode.use_inf_as_na = True


In [76]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
# standard code-template imports
from ta_lib.core.api import (
    create_context, get_dataframe, get_feature_names_from_column_transformer, get_package_path,
    display_as_tabs, string_cleaning, merge_info, initialize_environment,
    list_datasets, load_dataset, save_dataset
)
import ta_lib.eda.api as eda

In [78]:
import warnings

warnings.filterwarnings('ignore', message="The default value of regex will change from True to False in a future version.", 
                        category=FutureWarning)

In [79]:
initialize_environment(debug=False, hide_warnings=True)

## Utility functions

# 1. Initialization

In [80]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)
pprint(list_datasets(context))

housing_df = load_dataset(context, 'raw/housing')

['/raw/housing',
 '/raw/orders',
 '/raw/product',
 '/cleaned/housing',
 '/cleaned/orders',
 '/cleaned/product',
 '/cleaned/sales',
 '/processed/housing',
 '/processed/sales',
 '/train/housing/features',
 '/train/housing/target',
 '/train/sales/features',
 '/train/sales/target',
 '/test/housing/features',
 '/test/housing/target',
 '/test/sales/features',
 '/test/sales/target',
 '/score/housing/output',
 '/score/sales/output']


# 2. Data cleaning and consolidation

**<u>NOTES</u>**

The focus here is to create a cleaned dataset that is appropriate for solving the DS problem at hand from the raw data.

**1. Do**
* clean dataframe column names
* ensure dtypes are set properly
* join with other tables etc to create features
* transform, if appropriate, datetime like columns to generate additional features (weekday etc)
* transform, if appropriate, string columns to generate additional features
* discard cols that are not useful for training the model (IDs, constant cols, duplicate cols etc)
* additional features generated from existing columns


**2. Don't**
* handle missing values or outliers here. mark them and leave them for processing downstream.


## 2.1 Clean individual tables 

### Housing Table

In [81]:
eda.get_variable_summary(housing_df)

,Variable Name,Datatype,No of Unique,Samples
longitude,longitude,float64,844,"[-122.23, -122.22, -122.24, -122.25, -122.26]"
latitude,latitude,float64,862,"[37.88, 37.86, 37.85, 37.84, 37.83]"
housing_median_age,housing_median_age,float64,52,"[41.0, 21.0, 52.0, 42.0, 50.0]"
total_rooms,total_rooms,float64,5926,"[880.0, 7099.0, 1467.0, 1274.0, 1627.0]"
total_bedrooms,total_bedrooms,float64,1923,"[129.0, 1106.0, 190.0, 235.0, 280.0]"
population,population,float64,3888,"[322.0, 2401.0, 496.0, 558.0, 565.0]"
households,households,float64,1815,"[126.0, 1138.0, 177.0, 219.0, 259.0]"
median_income,median_income,float64,12928,"[8.3252, 8.3014, 7.2574, 5.6431, 3.8462]"
median_house_value,median_house_value,float64,3842,"[452600.0, 358500.0, 352100.0, 341300.0, 34220..."
ocean_proximity,ocean_proximity,object,5,"[NEAR BAY, <1H OCEAN, INLAND, NEAR OCEAN, ISLAND]"


In [82]:
# column names after cleaning

str_cols = list(
    set(housing_df.select_dtypes('object').columns.to_list()) 
)
housing_df_clean = (
    housing_df
    
    .copy()

    # set dtypes
    .change_type(['housing_median_age', 'total_rooms', 'population', 'households'], np.int64)
    
    # clean string columns
    .transform_columns(str_cols, string_cleaning, elementwise=False)

    # clean column names                                                                                                                                   
    .clean_names(case_type='snake')
)
housing_df_clean.head().T


,0,1,2,3,4
longitude,-122.23,-122.22,-122.24,-122.25,-122.25
latitude,37.88,37.86,37.85,37.85,37.85
housing_median_age,41,21,52,52,52
total_rooms,880,7099,1467,1274,1627
total_bedrooms,129.0,1106.0,190.0,235.0,280.0
population,322,2401,496,558,565
households,126,1138,177,219,259
median_income,8.3252,8.3014,7.2574,5.6431,3.8462
median_house_value,452600.0,358500.0,352100.0,341300.0,342200.0
ocean_proximity,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY


In [83]:
save_dataset(context, housing_df_clean, 'cleaned/housing')

## 2.2 Process tables 

In [84]:
# create a sample dataframe with minimal processing


# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(strategy="median")

# housing_num = housing_df_clean.drop("ocean_proximity", axis=1)

# imputer.fit(housing_num)
# X = imputer.transform(housing_num)

# housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_df_clean.index)
# housing_tr["rooms_per_household"] = (
#     housing_tr["total_rooms"] / housing_tr["households"]
# )
# housing_tr["bedrooms_per_room"] = (
#     housing_tr["total_bedrooms"] / housing_tr["total_rooms"]
# )
# housing_tr["population_per_household"] = (
#     housing_tr["population"] / housing_tr["households"]
# )

# housing_cat = housing_df_clean[["ocean_proximity"]]
# housing_df_processed = housing_tr.join(
#     pd.get_dummies(housing_cat, drop_first=True)
# )

In [85]:
housing_df_processed=housing_df_clean

In [86]:
# Any verifications on the data
from ta_lib.eda.api import get_variable_summary
display_as_tabs([
    ("Summary", f"Length: {len(housing_df_processed)}, Columns: {len(housing_df_processed.columns)}"),
    ("Variable summary", get_variable_summary(housing_df_processed)),
    ("head", housing_df_clean.head(5).T),
    ("tail", housing_df_clean.tail(5).T),
])

tzname OCEAN identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.


Tabs
    [0] Markdown(str, name='Summary')
    [1] DataFrame(height=300, name='Variable summary', sizing_mode='fixed', value=                         V..., width=800)
    [2] DataFrame(height=300, name='head', sizing_mode='fixed', value=              ..., width=800)
    [3] DataFrame(height=300, name='tail', sizing_mode='fixed', value=              ..., width=800)

In [87]:
save_dataset(context, housing_df_processed, 'cleaned/housing')

# 3. Generate Train, Validation and Test datasets



- We split the data into train, test (optionally, also a validation dataset)
- In this example, we are binning the target into 10 quantiles and then use a Stratified Shuffle to split the data.
- See sklearn documentation on the various available splitters
- https://scikit-learn.org/stable/modules/classes.html#splitter-classes
- This will go into production code (training only)

In [88]:
housing_df_processed.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41,880,129.0,322,126,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21,7099,1106.0,2401,1138,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52,1467,190.0,496,177,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52,1274,235.0,558,219,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52,1627,280.0,565,259,3.8462,342200.0,NEAR BAY


In [89]:
from sklearn.model_selection import StratifiedShuffleSplit
from ta_lib.core.api import custom_train_test_split  # helper function to customize splitting
from scripts import *

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=context.random_seed)
housing_df_train, housing_df_test = custom_train_test_split(housing_df_processed, splitter, by=binned_median_house_value)

In [90]:
target_col = "median_house_value"

train_X, train_y = (
    housing_df_train
    
    # split the dataset to train and test
    .get_features_targets(target_column_names=target_col)
)
save_dataset(context, train_X, 'train/housing/features')
save_dataset(context, train_y, 'train/housing/target')


test_X, test_y = (
    housing_df_test
    
    # split the dataset to train and test
    .get_features_targets(target_column_names=target_col)
)
save_dataset(context, test_X, 'test/housing/features')
save_dataset(context, test_y, 'test/housing/target')